In [1]:
import os
import glob
import json
import random
import numpy as np
import pandas as pd

In [ ]:
import utils
from scipy.io.wavfile import write
import soundfile as sf

import transforms
# Background - development - evaluation
# Development (enrollment) - Testing (Verification)

In [ ]:
a = utils.load_audio(path='data/loop3/0_0_0/01/0_0_0_0_0_0_.wav', sample_rate=16000)

In [ ]:
data={}
data['samples'] = a[0]
data['sample_rate'] = 16000

In [ ]:
audio_cfs = {
  # Init
  'sample_rate': 16000,
  'duration': 3, # second
  'add_sample': 240 #
  }
noise_path = {
    'rir_path': '../dataset/rir_data/RIRS_NOISES/simulated_rirs',
  'musan_path': '../dataset/musan_data/musan'}

In [ ]:
train_transform = transforms.build_transform(audio_config=audio_cfs,mode='train',num_stack=1,
noise_path=noise_path,stage=2)

In [ ]:
data_2 = train_transform(data)

In [ ]:
data['samples'].shape

In [ ]:
import torchaudio
import torch

In [ ]:
a = data['input'].expand(1,16000*3+240)

In [ ]:
torchaudio.save('test4.wav',a, 16000)

In [ ]:
a[0]

In [ ]:
data['samples']

In [ ]:
audio = np.fromstring(data['samples'],dtype=np.float64)

In [ ]:
audio.shape

In [ ]:
audio.astype(np.int16)

In [ ]:
write('test.wav',16000,audio.astype(np.int16))

In [ ]:
data['samples'].shape

In [ ]:
isinstance(data['samples'], np.float128)

In [ ]:
data['samples'].shape

In [ ]:
data_2['samples'].shape

In [2]:
info = json.load(open('meta_data/stage2/split_0.5_stage2.json'))

In [3]:
info

{'development': {'male': ['54',
   '51',
   '14',
   '03',
   '11',
   '22',
   '10',
   '40',
   '34',
   '45',
   '02',
   '17',
   '41',
   '55',
   '23',
   '05',
   '53',
   '21',
   '30',
   '44',
   '13',
   '20',
   '39',
   '09'],
  'female': ['26', '56', '59', '60', '58', '28']},
 'evaluation': {'male': ['31',
   '01',
   '33',
   '35',
   '08',
   '18',
   '16',
   '46',
   '50',
   '49',
   '27',
   '06',
   '15',
   '19',
   '48',
   '37',
   '42',
   '24',
   '04',
   '32',
   '38',
   '07',
   '29',
   '25'],
  'female': ['47', '57', '36', '43', '52', '12']}}

In [4]:
np.random.seed(2023)
random.seed(2023)

In [5]:
# LOOP = 5
WORD = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
DIR = './data'
# WORD = os.listdir('./data/experiments/')
# WORD

In [6]:
def prepare_data_experiment(root_dir,info):
    pattern = '%s/%s/%s/%s'
    for word in WORD:
        background = {
        'file': [],
        'speaker': [],
        'gender': [],
        'word': [], 
        }
        pairs = {
            'eval': {
                'female':[],
                'male':[],
                'all':[],
            }
        }
        all_speakers = set(info['evaluation']['female'] + info['evaluation']['male'] + info['development']['female'] + info['development']['male'])
        folder_word = word
        for phase in info:
            speakers = info[phase]
            for gender in speakers:
                if phase == 'development':
                    for spkr in speakers[gender]:
                        path = os.path.join(root_dir,'experiments', folder_word , spkr)
                        for file in os.listdir(path):
                            background['file'].append(pattern %('experiments',folder_word, spkr, file))
                            background['speaker'].append(spkr)
                            background['gender'].append(gender)
                            background['word'].append(folder_word)
                else:
                    for spkr in speakers[gender]:
                        imposters = set(info['evaluation'][gender] + info['development'][gender]) - set([spkr])
                        imposter_files = []
                        for imp in imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/*/%s/*.wav')%(2, imp)) + glob.glob(root_dir+('/experiments/*/%s/*.wav')%(imp))
                            imposter_files.extend(imp_file)

                        # Get all 
                        all_imposters = all_speakers - set([spkr])
                        all_imposter_files = []
                        for imp in all_imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/*/%s/*.wav')%(2, imp)) + glob.glob(root_dir+('/experiments/*/%s/*.wav')%(imp))
                            all_imposter_files.extend(imp_file)

                        path = os.path.join(root_dir, 'experiments',folder_word , spkr)
                        files = os.listdir(path)
                        for file in files:
                            current_file = pattern % ('experiments', folder_word, spkr, file)
                            # Get target correct
                            # Accept pairs
                            other_files = [f for f in files if f != file]
                            accept_pairs = [['1', current_file, pattern % ('experiments', folder_word, spkr, o_file)] for o_file in other_files]

                            # Reject pairs
                            reject_pairs = []
                            all_reject_pairs = []
                            n_pairs = len(files)
                            n_target_wrong = n_pairs // 2
                            n_imposters = n_pairs - n_target_wrong
                            
                            tw_all =list(set(glob.glob(root_dir+'/loop%s/*/%s/*.wav' % (2, spkr)) +glob.glob(root_dir+'/%s/*/%s/*.wav' % ('experiments', spkr))) -
                                set(glob.glob(root_dir+'/%s/%s/%s/*.wav' % ('experiments', folder_word, spkr))))
                            tw_set = random.choices(tw_all, k=n_target_wrong)
                            for tw_sample in tw_set:
                                reject_pairs.append(['0', current_file, tw_sample[len(root_dir)+1:]])
                                all_reject_pairs.append(['0', current_file, tw_sample[len(root_dir)+1:]])

                            imposter_set = random.choices(imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])
                            pairs['eval'][gender].extend([accept_pairs] + [reject_pairs])


                            
                            imposter_set = random.choices(all_imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                all_reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])
                            pairs['eval']['all'].extend([accept_pairs] + [all_reject_pairs])

                            
        # df_bg = pd.DataFrame(background)
        # df_bg.to_csv(os.path.join(root_dir,'%s/%s/development.csv'%('experiments',folder_word)), index=False)
        print(word)
        for phase in pairs:
            files = pairs[phase]
            for gender in files:
                fout = open(os.path.join(root_dir,'%s/%s/%s_%s_2.txt'%('experiments',folder_word,gender,phase)), 'w')
                for list_file in files[gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')
  


In [7]:
prepare_data_experiment(DIR, info)

9_1
9_6
1_6
1_9
6_3
6_1
3_9
9_3
3_1
1_3
6_9
3_6


In [7]:
def prepare_data_stage2_2(root_dir,info,loop):
    pattern = 'loop%s/%s/%s/%s'
    all_speakers = set(info['evaluation']['female'] + info['evaluation']['male'] + info['development']['female'] + info['development']['male'])
    for word in WORD:
        background = {
        'file': [],
        'speaker': [],
        'gender': [],
        'word': [], 
        }
        pairs = {
            'eval': {
                'female':[],
                'male':[],
                'all':[],
            }
        }
        folder_word = word + ('_'+word)*(loop-1)
        for phase in info:
            speakers = info[phase]
            for gender in speakers:
                if phase == 'development':
                    for spkr in speakers[gender]:
                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        for file in os.listdir(path):
                            background['file'].append(pattern %(loop, folder_word, spkr, file))
                            background['speaker'].append(spkr)
                            background['gender'].append(gender)
                            background['word'].append(folder_word)
                else:
                    for spkr in speakers[gender]:
                        imposters = set(info['evaluation'][gender] + info['development'][gender]) - set([spkr])
                        imposter_files = []
                        for imp in imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/*/%s/*.wav')%(loop, imp))
                            imposter_files.extend(imp_file)

                        # Get all 
                        all_imposters = all_speakers - set([spkr])
                        all_imposter_files = []
                        for imp in all_imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/*/%s/*.wav')%(loop, imp))
                            all_imposter_files.extend(imp_file)

                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        files = os.listdir(path)
                        for file in files:
                            current_file = pattern % (loop, folder_word, spkr, file)
                            # Get target correct
                            # Accept pairs
                            other_files = [f for f in files if f != file]
                            accept_pairs = [['1', current_file, pattern % (loop, folder_word, spkr, o_file)] for o_file in other_files]

                            # Reject pairs
                            reject_pairs = []
                            all_reject_pairs = []
                            n_pairs = len(files)
                            n_target_wrong = n_pairs // 2
                            n_imposters = n_pairs - n_target_wrong
                            
                            tw_all =list(set(glob.glob(root_dir+'/loop%s/*/%s/*.wav' % (loop, spkr))) -
                                set(glob.glob(root_dir+'/loop%s/%s/%s/*.wav' % (loop, folder_word, spkr))))
                            tw_set = random.choices(tw_all, k=n_target_wrong)
                            for tw_sample in tw_set:
                                reject_pairs.append(['0', current_file, tw_sample[len(root_dir)+1:]])
                                all_reject_pairs.append(['0', current_file, tw_sample[len(root_dir)+1:]])
                            imposter_set = random.choices(imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])
                            pairs['eval'][gender].extend([accept_pairs] + [reject_pairs])

                            
                            imposter_set = random.choices(all_imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                all_reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])
                            pairs['eval']['all'].extend([accept_pairs] + [all_reject_pairs])


        # df_bg = pd.DataFrame(background)
        # df_bg.to_csv(os.path.join(root_dir,'loop%s/%s/development.csv'%(loop,folder_word)), index=False)
        print(folder_word)
        for phase in pairs:
            files = pairs[phase]
            for gender in files:
                fout = open(os.path.join(root_dir,'loop%s/%s/%s_%s_2.txt'%(loop,folder_word,gender,phase)), 'w')
                for list_file in files[gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')
            
            # files = pairs[phase]['male'] + pairs[phase]['female']
            # fout = open(os.path.join(root_dir,'loop%s/%s/all_%s.txt'%(loop,folder_word,phase)), 'w')
            # for list_file in files:
            #     for file in list_file:
            #         fout.write(" ".join(file) + '\n')
        

        


In [12]:
prepare_data_stage2_2('data', info, loop=5)

0_0_0_0_0
1_1_1_1_1
2_2_2_2_2
3_3_3_3_3
4_4_4_4_4
5_5_5_5_5
6_6_6_6_6
7_7_7_7_7
8_8_8_8_8
9_9_9_9_9


In [ ]:
def prepare_data_stage2(root_dir,info,loop):
    pattern = 'loop%s/%s/%s/%s'
    for word in WORD:
        background = {
        'file': [],
        'speaker': [],
        'gender': [],
        'word': [], 
        }
        pairs = {
            'development': {
                'female':[],
                'male': [],
            },
            'evaluation': {
                'female':[],
                'male':[],
            }
        }
        folder_word = word + ('_'+word)*(loop-1)
        for phase in info:
            speakers = info[phase]
            for gender in speakers:
                if phase == 'development':
                    for spkr in speakers[gender]:
                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        for file in os.listdir(path):
                            background['file'].append(pattern %(loop, folder_word, spkr, file))
                            background['speaker'].append(spkr)
                            background['gender'].append(gender)
                            background['word'].append(folder_word)
                else:
                    for spkr in speakers[gender]:
                        imposters = set(info['development'][gender] )+ set(info['evaluation'][gender]) - set([spkr])
                        imposter_files = []
                        for imp in imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/*/%s/*.wav')%(loop, imp))
                            imposter_files.extend(imp_file)

                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        files = os.listdir(path)
                        for file in files:
                            current_file = pattern % (loop, folder_word, spkr, file)
                            # Get target correct
                            # Accept pairs
                            other_files = [f for f in files if f != file]
                            accept_pairs = [['1', current_file, pattern % (loop, folder_word, spkr, o_file)] for o_file in other_files]

                            # Reject pairs
                            reject_pairs = []
                            n_pairs = len(files)
                            n_target_wrong = n_pairs // 2
                            n_imposters = n_pairs - n_target_wrong
                            
                            tw_all =list(set(glob.glob(root_dir+'/loop%s/*/%s/*.wav' % (loop, spkr))) -
                                set(glob.glob(root_dir+'/loop%s/%s/%s/*.wav' % (loop, folder_word, spkr))))
                            tw_set = random.choices(tw_all, k=n_target_wrong)
                            for tw_sample in tw_set:
                                reject_pairs.append(['0', current_file, tw_sample[len(root_dir)+1:]])

                            imposter_set = random.choices(imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])

                            
                            pairs[phase][gender].extend([accept_pairs] + [reject_pairs])

        # df_bg = pd.DataFrame(background)
        # df_bg.to_csv(os.path.join(root_dir,'loop%s/%s/background.csv'%(loop,folder_word)), index=False)

        for phase in pairs:
            files = pairs[phase]
            for gender in files:
                fout = open(os.path.join(root_dir,'loop%s/%s/%s_%s.txt'%(loop,folder_word,gender,phase)), 'w')
                for list_file in files[gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')

        


In [ ]:
def prepare_pair_enrollment(root_dir,info,loop):
    pattern = 'loop%s/%s/%s/%s'
    for word in WORD:
        pairs = {
            'dev': {
                'female':[],
                'male': [],
            }
        }
        folder_word = word + ('_'+word)*(loop-1)
        for phase in info:
            speakers = info[phase]
            for gender in speakers:
                if phase == 'development':
                    for spkr in speakers[gender]:
                        imposters = set(info['development']['female'] + info['development']['male']) - set([spkr])
                        imposter_files = []
                        for imp in imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/*/%s/*.wav')%(loop, imp))
                            imposter_files.extend(imp_file)

                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        files = os.listdir(path)
                        for file in files:
                            current_file = pattern % (loop, folder_word, spkr, file)
                            # Get target correct
                            # Accept pairs
                            other_files = [f for f in files if f != file]
                            accept_pairs = [['1', current_file, pattern % (loop, folder_word, spkr, o_file)] for o_file in other_files]

                            # Reject pairs
                            reject_pairs = []
                            n_pairs = len(files)
                            n_target_wrong = n_pairs // 2
                            n_imposters = n_pairs - n_target_wrong
                            
                            tw_all =list(set(glob.glob(root_dir+'/loop%s/*/%s/*.wav' % (loop, spkr))) -
                                set(glob.glob(root_dir+'/loop%s/%s/%s/*.wav' % (loop, folder_word, spkr))))
                            tw_set = random.choices(tw_all, k=n_target_wrong)
                            for tw_sample in tw_set:
                                reject_pairs.append(['0', current_file, tw_sample[len(root_dir)+1:]])

                            imposter_set = random.choices(imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])

                            
                            pairs['dev'][gender].extend([accept_pairs] + [reject_pairs])
                    

        # df_bg = pd.DataFrame(background)
        # df_bg.to_csv(os.path.join(root_dir,'loop%s/%s/development.csv'%(loop,folder_word)), index=False)

        for phase in pairs:
            files = pairs[phase]
            for gender in files:
                fout = open(os.path.join(root_dir,'loop%s/%s/%s_%s.txt'%(loop,folder_word,gender,phase)), 'w')
                for list_file in files[gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')
            
            files = pairs[phase]['male'] + pairs[phase]['female']
            fout = open(os.path.join(root_dir,'loop%s/%s/all_%s.txt'%(loop,folder_word,phase)), 'w')
            for list_file in files:
                for file in list_file:
                    fout.write(" ".join(file) + '\n')
        


In [ ]:
prepare_pair_enrollment(DIR, info, 5)

In [ ]:
DIR = 'new_data'

In [ ]:
prepare_data_stage2_2(DIR, info, 5)

In [ ]:
set(glob.glob('./data'+'/loop2/*/01/*.wav')) - set(glob.glob('./data'+'/loop2/0_0/01/*.wav'))

In [ ]:
# import required libraries
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
 
# Sampling frequency
freq = 44100
 
# Recording duration
duration = 5
 
# Start recorder with the given values
# of duration and sample frequency
recording = sd.rec(int(duration * freq),
                   samplerate=freq, channels=2)
 
# Record audio for the given number of seconds
sd.wait()
 
# This will convert the NumPy array to an audio
# file with the given sampling frequency
write("recording0.wav", freq, recording)
 
# Convert the NumPy array to audio file
wv.write("recording1.wav", recording, freq, sampwidth=2)